In [104]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics
import seaborn as sns

sns.set()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
print(Candledf.tail())

                ndatetime   open   high    low  close  volume
10086 2021-11-03 10:57:20  17061  17091  17040  17091   12000
10087 2021-11-03 11:30:13  17091  17106  17076  17085   12000
10088 2021-11-03 12:18:37  17085  17116  17080  17087   12000
10089 2021-11-03 13:11:59  17087  17106  17085  17094   12000
10090 2021-11-03 13:44:50  17092  17093  17090  17092     152


In [105]:
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm

In [106]:
Candledf['Y'] = Candledf.close.shift(-1)
print(Candledf.Y.tail(5))
Candledf.iloc[:,1:4] = normalize(Candledf.iloc[:,1:4])
Candledf.head()

10086    17085.0
10087    17087.0
10088    17094.0
10089    17092.0
10090        NaN
Name: Y, dtype: float64


,ndatetime,open,high,low,close,volume,Y
0,2019-03-20 15:00:00.000000,0.227123,0.223828,0.229712,10514,12000,10513.0
1,2019-03-20 19:35:12.000000,0.226401,0.222899,0.228994,10513,12000,10488.0
2,2019-03-20 23:01:53.000000,0.226298,0.222280,0.226942,10488,12000,10542.0
3,2019-03-21 01:21:39.000300,0.223722,0.225377,0.227660,10542,12000,10506.0
4,2019-03-21 02:30:19.000300,0.229287,0.226925,0.229096,10506,12000,10501.0


In [107]:
def train_windows(df, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["Y"]))
    return np.array(X_train), np.array(Y_train)

In [108]:
test = Candledf[-30:]
stock = Candledf[:10060]
test.reset_index()
stock.reset_index()

,index,ndatetime,open,high,low,close,volume,Y
0,0,2019-03-20 15:00:00.000000,0.227123,0.223828,0.229712,10514,12000,10513.0
1,1,2019-03-20 19:35:12.000000,0.226401,0.222899,0.228994,10513,12000,10488.0
2,2,2019-03-20 23:01:53.000000,0.226298,0.222280,0.226942,10488,12000,10542.0
3,3,2019-03-21 01:21:39.000300,0.223722,0.225377,0.227660,10542,12000,10506.0
4,4,2019-03-21 02:30:19.000300,0.229287,0.226925,0.229096,10506,12000,10501.0
...,...,...,...,...,...,...,...,...
10055,10055,2021-11-01 09:59:57.000000,0.901793,0.900268,0.900893,17052,12000,17030.0
10056,10056,2021-11-01 10:18:50.000000,0.899938,0.899752,0.898738,17030,12000,17086.0
10057,10057,2021-11-01 10:42:37.000000,0.897980,0.901301,0.898738,17086,12000,17061.0
10058,10058,2021-11-01 11:07:38.000000,0.903648,0.901611,0.901611,17061,12000,17090.0


In [109]:
stock = stock[['close','Y']]
test = test[['close','Y']]
stock.tail()

,close,Y
10055,17052,17030.0
10056,17030,17086.0
10057,17086,17061.0
10058,17061,17090.0
10059,17090,17076.0


In [110]:
X_train, Y_train = train_windows(stock, 5, 1)
X_test, Y_test = train_windows(test, 5, 1)

In [111]:
class stock_lstm(tf.keras.Model):
  def __init__(self,units):
      super(stock_lstm, self).__init__()
      self.rnn = tf.keras.Sequential([tf.keras.layers.SimpleRNN(units,input_shape=(5,2),dropout=0.2,return_sequences=True,unroll=True),
          tf.keras.layers.SimpleRNN(units,dropout=0.2,unroll=True)])
      self.fc = tf.keras.layers.Dense(32)
      self.out = tf.keras.layers.Dense(1)
  def call(self, inputs, training=None):
      x = self.rnn(inputs)
      x = self.fc(x)
      out = self.out(x)
      return out

In [112]:
batchsz = 16

data = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
data = data.batch(batchsz,drop_remainder=True)


data_test = tf.data.Dataset.from_tensor_slices((X_test,Y_test))
data_test = data_test.batch(batchsz,drop_remainder=True)


In [113]:
data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [ ]:
units = 128
epochs = 10


rnn_model = stock_lstm(units)
rnn_model.compile(optimizer  =keras.optimizers.Adam(),loss="mean_absolute_error",metrics=['mean_absolute_error'])
rnn_model.fit(data,epochs=epochs, validation_data = data_test,shuffle=True)